# LCEL custom 

In [5]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings 
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.output_parsers import StrOutputParser
from langsmith.run_helpers import traceable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

import sys
sys.path.append("../../")
from src.config import Configuration
from src.prepare.data_load import DocDataLoader

class HybridGeminiRagLearn:

    def __init__(self, es_index: str, embed_model: str, config: Configuration):
        """
            Initialize the RAG
            @param es_index elastic search index
            @param embed_model hugging face sentence-transformer model
            @param config Configuration
        """
        self.chat_model = ChatGoogleGenerativeAI(
            model="gemini-pro", 
            temperature=0, 
            google_api_key=config.load_gemini_token()
        )
        template = """\
            Answer the question based only on the following context:\n \
                {context} \
            \nQuestion: {question}"""
        prompt = ChatPromptTemplate.from_template(template)
        self._init_retriever(es_index=es_index, embed_model=embed_model, config=config)
        self.rag = (
            {"context": self.ensemble_retriever, "question": RunnablePassthrough()}
            | RunnableLambda(data)
            | prompt 
        ) 
        return
    
    @traceable(tags=['learn', "custom-lcel"])
    def run(self, input):
        return self.rag.invoke(input)

    def _init_retriever(self, es_index:str, embed_model:str, config:Configuration):
        """
            Initialize Elastic, BM25 instances and combine them into hybrid search
            This method is intented for private use. Please be cautious modifying.
        """
        # embeddings = HuggingFaceInferenceAPIEmbeddings(
        #     model_name=embed_model, api_key=config.load_hg_token(nth=0))
        embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=config.load_gemini_token())
        # elastic
        es = ElasticsearchStore(
            es_connection=config.load_elasticsearch_connection(),
            embedding=embeddings,
            index_name=es_index, distance_strategy="EUCLIDEAN_DISTANCE") 
        self.es_retriever = es.as_retriever(search_kwargs={"k": 2})
        # BM-25
        doc_loader = DocDataLoader()
        self.bm25_retriever = BM25Retriever.from_documents(doc_loader.load_major_docs())
        self.bm25_retriever.k = 2
        self.ensemble_retriever = EnsembleRetriever(
            retrievers=[self.bm25_retriever, self.es_retriever],
            weights=[0.5, 0.5])
        del doc_loader
        return self.ensemble_retriever

In [6]:
config = Configuration()
hybrid = HybridGeminiRagLearn(es_index="labse-major", embed_model="sentence-transformer/LaBSE", config=config)

From (44, 4) to Counter({1: 44})


In [7]:
config.enable_tracing(project="LEARN")
a = hybrid.run("hãy tư vấn ngành Dược")
a

ChatPromptValue(messages=[HumanMessage(content="            Answer the question based only on the following context:\n                 [Document(page_content='Công nghệ sinh học là ngành ứng dụng các kiến thức trong lĩnh vực sinh học nhằm nghiên cứu và sản xuất ra các sản phẩm sinh học thiết yếu, sản phẩm có giá trị cao như vaccine, kháng sinh, xăng sinh học; cải tiến chất lượng giống cây trồng, ứng dụng vi sinh để xử lý nước thải công nghiệp, ứng dụng các hợp chất tự nhiên để điều trị bệnh và những ứng dụng khác phục vụ cho đời sống con người.\\n\\nVới cách tiếp cận và xây dựng chương trình đào tạo hiện đại, tiệm cận với các nền giáo dục tiên tiến và sát với yêu cầu về nguồn nhân lực của thị trường lao động, sinh viên sau khi tốt nghiệp ngành Công nghệ sinh học có thể đảm nhiệm các vị trí như: Phụ trách nghiên cứu và phát triển sản phẩm mới; phụ trách quản lý hệ thống và kiểm định dây chuyền sản xuất của nhà máy; phụ trách theo dõi và kiểm soát vi sinh sản phẩm; chẩn đoán bệnh bằng

In [11]:
a.to_string()

"Human:             Answer the question based only on the following context:\n                 [Document(page_content='Công nghệ sinh học là ngành ứng dụng các kiến thức trong lĩnh vực sinh học nhằm nghiên cứu và sản xuất ra các sản phẩm sinh học thiết yếu, sản phẩm có giá trị cao như vaccine, kháng sinh, xăng sinh học; cải tiến chất lượng giống cây trồng, ứng dụng vi sinh để xử lý nước thải công nghiệp, ứng dụng các hợp chất tự nhiên để điều trị bệnh và những ứng dụng khác phục vụ cho đời sống con người.\\n\\nVới cách tiếp cận và xây dựng chương trình đào tạo hiện đại, tiệm cận với các nền giáo dục tiên tiến và sát với yêu cầu về nguồn nhân lực của thị trường lao động, sinh viên sau khi tốt nghiệp ngành Công nghệ sinh học có thể đảm nhiệm các vị trí như: Phụ trách nghiên cứu và phát triển sản phẩm mới; phụ trách quản lý hệ thống và kiểm định dây chuyền sản xuất của nhà máy; phụ trách theo dõi và kiểm soát vi sinh sản phẩm; chẩn đoán bệnh bằng công nghệ di truyền, liệu pháp gene, cô